In [1]:
# Library
import sys
import numpy as np
import pandas as pd
import random
import os
import glob
import warnings
warnings.filterwarnings("ignore")

from bayes_opt import BayesianOptimization
from catboost import CatBoostRegressor

In [2]:
print(f'python : {sys.version}')
print(f'numpy : {np.__version__}')
print(f'pandas : {pd.__version__}')

python : 3.6.9 (default, Oct  8 2020, 12:12:24) 
[GCC 8.4.0]
numpy : 1.19.5
pandas : 1.1.5


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/DACON/태양광/train_reshape.csv')
test_df = pd.read_csv('/content/drive/MyDrive/DACON/태양광/test_reshape.csv')


def target_scaler(x):
  for i in range(200):
    if (x>i) & (x<=i+1):
      return i
    elif x==0:
      return 0

train_df['TARGET_day7_cata']=train_df['TARGET_day7'].apply(target_scaler)


from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
train_df.TARGET_day7_cata = label_encoder.fit_transform(train_df.TARGET_day7_cata)

from sklearn.utils.multiclass import type_of_target
print(type_of_target(train_df.TARGET_day7_cata))

train_df1=train_df.iloc[:48*365,:]
train_df2=train_df.iloc[48*365:365*48*2,:]
train_df3=train_df.iloc[365*48*2:,:]

from sklearn.model_selection import StratifiedKFold
stf=StratifiedKFold(n_splits=5, shuffle=False, random_state=100)

#마지막 세트를 사용
for train_index1,valid_index1 in stf.split(train_df1, train_df1.TARGET_day7_cata):
  print(train_index1, valid_index1)

for train_index2,valid_index2 in stf.split(train_df2, train_df2.TARGET_day7_cata):
  print(train_index2, valid_index2)

for train_index3,valid_index3 in stf.split(train_df3, train_df3.TARGET_day7_cata):
  print(train_index3, valid_index3)

multiclass
[ 1997  2084  2089 ... 17517 17518 17519] [   0    1    2 ... 7077 7514 7558]
[    0     1     2 ... 17517 17518 17519] [1997 2084 2089 ... 8707 8769 9055]
[    0     1     2 ... 17517 17518 17519] [ 5402  5403  5680 ... 12353 12366 12416]
[    0     1     2 ... 17517 17518 17519] [ 7272  7752  7798 ... 15719 15766 15910]
[    0     1     2 ... 15719 15766 15910] [ 7800  7801  7945 ... 17517 17518 17519]
[ 2235  2322  2465 ... 17517 17518 17519] [   0    1    2 ... 6887 6933 6981]
[    0     1     2 ... 17517 17518 17519] [2235 2322 2465 ... 8844 8892 8940]
[    0     1     2 ... 17517 17518 17519] [ 4493  4877  5068 ... 12687 12943 12983]
[    0     1     2 ... 17517 17518 17519] [ 7422  7607  7656 ... 15957 16006 16153]
[    0     1     2 ... 15957 16006 16153] [ 7608  7609  7657 ... 17517 17518 17519]
[ 1611  1612  1702 ... 17133 17134 17135] [   0    1    2 ... 6411 6552 7367]
[    0     1     2 ... 17133 17134 17135] [1611 1612 1702 ... 8817 8867 9036]
[    0     1     

In [ ]:
train_df2 = train_df2.reset_index(drop=True)
train_df3 = train_df3.reset_index(drop=True)

del train_df1['TARGET_day7_cata']
del train_df2['TARGET_day7_cata']
del train_df3['TARGET_day7_cata']

X_train_1 = train_df1[train_df1.index.isin(train_index1)].drop(columns=['TARGET_day7','TARGET_day8'])
Y_train_1 = train_df1[train_df1.index.isin(train_index1)]['TARGET_day7']
X_valid_1 = train_df1[train_df1.index.isin(valid_index1)].drop(columns=['TARGET_day7','TARGET_day8'])
Y_valid_1 = train_df1[train_df1.index.isin(valid_index1)]['TARGET_day7']

X_train_2 = train_df2[train_df2.index.isin(train_index2)].drop(columns=['TARGET_day7','TARGET_day8'])
Y_train_2 = train_df2[train_df2.index.isin(train_index2)]['TARGET_day7']
X_valid_2 = train_df2[train_df2.index.isin(valid_index2)].drop(columns=['TARGET_day7','TARGET_day8'])
Y_valid_2 = train_df2[train_df2.index.isin(valid_index2)]['TARGET_day7']

X_train_3 = train_df3[train_df3.index.isin(train_index3)].drop(columns=['TARGET_day7','TARGET_day8'])
Y_train_3 = train_df3[train_df3.index.isin(train_index3)]['TARGET_day7']
X_valid_3 = train_df3[train_df3.index.isin(valid_index3)].drop(columns=['TARGET_day7','TARGET_day8'])
Y_valid_3 = train_df3[train_df3.index.isin(valid_index3)]['TARGET_day7']

print(X_train_1.shape, Y_train_1.shape, X_valid_1.shape, Y_valid_1.shape)
print(X_train_2.shape, Y_train_2.shape, X_valid_2.shape, Y_valid_2.shape)
print(X_train_3.shape, Y_train_3.shape, X_valid_3.shape, Y_valid_3.shape)

(14016, 190) (14016,) (3504, 190) (3504,)
(14016, 190) (14016,) (3504, 190) (3504,)
(13709, 190) (13709,) (3427, 190) (3427,)


In [ ]:
quantiles = [0.1, 0.2, 0.3, 0.4, 0.5 ,0.6, 0.7, 0.8, 0.9]

for q in quantiles:
  def catboost_quantile_loss(X_train, y_train, X_valid, y_valid, loss_function='Quantile:alpha={}'.format(q),
                  iterations=500, learning_rate=0.05, depth=10, random_seed = 100,bagging_temperature = 0.2,
                  od_type='Iter', metric_period = 50, od_wait=20,min_data_in_leaf=32
                  ):
    
    
    catmodel = CatBoostRegressor( 
    loss_function= loss_function,
    iterations=iterations,
    learning_rate=learning_rate,
    depth=depth,
    random_seed=random_seed,
    bagging_temperature= bagging_temperature,
    od_type=od_type,
    metric_period=metric_period,
    min_data_in_leaf = min_data_in_leaf,        
    od_wait= 20,
    verbose = 100
    )

    catmodel.fit(X_train,
                 y_train,
                 eval_set=(X_valid,y_valid),
                 plot=False
            )

    predict = catmodel.predict(X_valid)
    
    best_quantile_loss_train = catmodel.best_score_['learn']['Quantile:alpha={}'.format(q)]
    best_quantile_loss_valid = catmodel.best_score_['validation']['Quantile:alpha={}'.format(q)]
        
        
    print('train_quantile_loss : {:.4f}, test_quantile_loss : {:.4f}'.format(best_quantile_loss_train, best_quantile_loss_valid))
    
    return - best_quantile_loss_valid

  def optimize_catboost(X_train,y_train,X_valid,y_valid, param_probe=None) :
    '''
    return the optimizer  and output the best hyperparameters found
    param_probe : dict of hyperparameters in order to guide the optimization
    '''
    def catboost_wrapper(learning_rate, depth, 
                         min_data_in_leaf, bagging_temperature,
                         loss_function = 'Quantile:alpha={}'.format(q), 
                         random_seed = 100,  iterations = 500) :
        '''
        wrapper for the function to optimize
        '''
        
        return catboost_quantile_loss(X_train=X_train, y_train=y_train ,
                             X_valid=X_valid ,y_valid=y_valid, 
                             iterations = int(iterations),
                             random_seed = random_seed, 
                             learning_rate=learning_rate, 
                             depth=int(depth),
                             min_data_in_leaf= int(min_data_in_leaf),
                             bagging_temperature = bagging_temperature
              )
    
    optimizer = BayesianOptimization(
        f = catboost_wrapper,
        pbounds= {
            "learning_rate" : (0.01,0.50),
            "depth" : (2,10),
            "min_data_in_leaf" : (1,10),
            "bagging_temperature" : (0,100),       
        },
        random_state = 100,
        verbose = 50,
    )
    if param_probe != None :
        optimizer.probe(
        params= param_probe,
        lazy=True,
        )
    
    optimizer.maximize(n_iter=5,
                      init_points=5)
    
    print("Final result:", optimizer.max)
    return optimizer

  print('-- Optimizing Quantile_{} Parameters --'.format(q))
  optimizer_cat = optimize_catboost(X_train=X_train_1, y_train=Y_train_1, 
                                  X_valid=X_valid_1, y_valid=Y_valid_1)
  
  params = optimizer_cat.max['params']
  params['depth'] = int(params['depth'])
  params['min_data_in_leaf'] = int(params['min_data_in_leaf'])
  
  cb_model = CatBoostRegressor(iterations=500,
                               random_seed = 100,
                             od_type='Iter',
                             metric_period = 50,
                             od_wait=20,
                             loss_function='Quantile:alpha={0}'.format(q),
                             **params
                             )
  
  print('---- Predict',q,'----')
  cb_model.fit(X_train_1, Y_train_1,
             eval_set=(X_valid_1, Y_valid_1),
             use_best_model=True,
             verbose=100)
  
  pred_test_cat = cb_model.predict(test_df)
  pred_test_cat = pd.DataFrame(pred_test_cat)
  pred_test_cat.to_csv('/content/drive/MyDrive/DACON/태양광/quantiles/catboost/year1/1_cat_quantile_{}.csv'.format(q), index=False)

-- Optimizing Quantile_0.1 Parameters --
|   iter    |  target   | baggin... |   depth   | learni... | min_da... |
-------------------------------------------------------------------------


0:	learn: 1.6743395	test: 1.6771366	best: 1.6771366 (0)	total: 71.9ms	remaining: 35.9s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 1.29755559
bestIteration = 47

Shrink model to first 48 iterations.
train_quantile_loss : 1.2894, test_quantile_loss : 1.2976
|  1        | -1.298    |  54.34    |  4.227    |  0.218    |  8.603    |


0:	learn: 1.6809065	test: 1.6775206	best: 1.6775206 (0)	total: 15.6ms	remaining: 7.79s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 1.31873269
bestIteration = 15

Shrink model to first 16 iterations.
train_quantile_loss : 1.6809, test_quantile_loss : 1.3187
|  2        | -1.319    |  0.4719   |  2.973    |  0.3387   |  8.433    |


0:	learn: 1.6111260	test: 1.6134266	best: 1.6134266 (0)	total: 46.8ms	remaining: 23.3s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 1.272255198
bestIteration = 22

Shrink model to first 23 iterations.
train_quantile_loss : 1.6111, test_quantile_loss : 1.2723
|  3        | -1.272    |  13.67    |  6.601    |  0.4467   |  2.883    |


0:	learn: 1.7090774	test: 1.7095730	best: 1.7095730 (0)	total: 13.9ms	remaining: 6.95s
100:	learn: 1.3325453	test: 1.3154464	best: 1.3118795 (95)	total: 1.18s	remaining: 4.66s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 1.311879527
bestIteration = 95

Shrink model to first 96 iterations.
train_quantile_loss : 1.3325, test_quantile_loss : 1.3119
|  4        | -1.312    |  18.53    |  2.867    |  0.1177   |  9.808    |
0:	learn: 1.6456791	test: 1.6470275	best: 1.6470275 (0)	total: 18.1ms	remaining: 9.02s


...

0:	learn: 4.5149147	test: 4.3169634	best: 4.3169634 (0)	total: 9.59ms	remaining: 4.78s
100:	learn: 0.9769221	test: 0.7881528	best: 0.7855251 (84)	total: 1.2s	remaining: 4.73s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.7855251324
bestIteration = 84

Shrink model to first 85 iterations.
train_quantile_loss : 0.9769, test_quantile_loss : 0.7855
|  7        | -0.7855   |  100.0    |  2.0      |  0.5      |  10.0     |


0:	learn: 4.5837101	test: 4.5130269	best: 4.5130269 (0)	total: 46ms	remaining: 22.9s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.7785505587
bestIteration = 50

Shrink model to first 51 iterations.
train_quantile_loss : 0.8557, test_quantile_loss : 0.7786
|  8        | -0.7786   |  11.08    |  7.073    |  0.3376   |  1.941    |


0:	learn: 5.7412896	test: 5.7395397	best: 5.7395397 (0)	total: 10.2ms	remaining: 5.1s
100:	learn: 3.5148937	test: 3.2571252	best: 3.2571252 (100)	total: 1.01s	remaining: 3.99s
200:	learn: 2.0501675	test: 1.7491159	best: 1.7491159 (200)	total: 2.11s	remaining: 3.13s
300:	learn: 1.3990725	test: 1.0970844	best: 1.0970844 (300)	total: 3.45s	remaining: 2.28s
400:	learn: 1.1550451	test: 0.8594685	best: 0.8594685 (400)	total: 4.81s	remaining: 1.19s
499:	learn: 1.0613086	test: 0.7657459	best: 0.7657459 (499)	total: 6.17s	remaining: 0us

bestTest = 0.7657458736
bestIteration = 499

train_quantile_loss : 1.0613, test_quantile_loss : 0.7657
|  9        | -0.7657   |  41.5     |  2.0      |  0.01     |  1.0      |


0:	learn: 5.7412896	test: 5.7395397	best: 5.7395397 (0)	total: 10.4ms	remaining: 5.2s
100:	learn: 3.5148937	test: 3.2571252	best: 3.2571252 (100)	total: 1.06s	remaining: 4.19s
200:	learn: 2.0501675	test: 1.7491159	best: 1.7491159 (200)	total: 2.12s	remaining: 3.16s
300:	learn: 1.3990725	test: 1.0970844	best: 1.0970844 (300)	total: 3.45s	remaining: 2.28s
400:	learn: 1.1550451	test: 0.8594685	best: 0.8594685 (400)	total: 4.82s	remaining: 1.19s
499:	learn: 1.0613086	test: 0.7657459	best: 0.7657459 (499)	total: 6.14s	remaining: 0us

bestTest = 0.7657458736
bestIteration = 499

train_quantile_loss : 1.0613, test_quantile_loss : 0.7657
|  10       | -0.7657   |  70.66    |  2.0      |  0.01     |  10.0     |
Final result: {'target': -0.6520701416659377, 'params': {'bagging_temperature': 54.340494179096545, 'depth': 4.226955080750369, 'learning_rate': 0.21801361946707523, 'min_data_in_leaf': 8.602985190879133}}
---- Predict 0.9 ----


0:	learn: 5.0080454	test: 4.8830973	best: 4.8830973 (0)	total: 15.9ms	remaining: 7.91s
100:	learn: 0.9037611	test: 0.6753257	best: 0.6753257 (100)	total: 1.98s	remaining: 7.81s
200:	learn: 0.8299646	test: 0.6558025	best: 0.6520701 (190)	total: 3.92s	remaining: 5.83s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.6520701417
bestIteration = 190

Shrink model to first 191 iterations.


In [ ]:
test_files  = sorted(glob.glob('/content/drive/MyDrive/DACON/태양광/quantiles/lgbm_cv/3year/*csv'))
test_files

['/content/drive/MyDrive/DACON/태양광/quantiles/lgbm_cv/3year/year3_results_0.09.csv',
 '/content/drive/MyDrive/DACON/태양광/quantiles/lgbm_cv/3year/year3_results_0.19.csv',
 '/content/drive/MyDrive/DACON/태양광/quantiles/lgbm_cv/3year/year3_results_0.29.csv',
 '/content/drive/MyDrive/DACON/태양광/quantiles/lgbm_cv/3year/year3_results_0.39.csv',
 '/content/drive/MyDrive/DACON/태양광/quantiles/lgbm_cv/3year/year3_results_0.5.csv',
 '/content/drive/MyDrive/DACON/태양광/quantiles/lgbm_cv/3year/year3_results_0.61.csv',
 '/content/drive/MyDrive/DACON/태양광/quantiles/lgbm_cv/3year/year3_results_0.71.csv',
 '/content/drive/MyDrive/DACON/태양광/quantiles/lgbm_cv/3year/year3_results_0.81.csv',
 '/content/drive/MyDrive/DACON/태양광/quantiles/lgbm_cv/3year/year3_results_0.91.csv']

In [ ]:
result_1 = []
result_1 = pd.DataFrame()

for i in range(0,len(test_files)):
  result_1 = pd.concat([result_1, pd.read_csv(test_files[i])], axis=1)

In [ ]:
result_1

,0.09,0.19,0.29,0.39,0.5,0.61,0.71,0.81,0.91
0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.03
1,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.01
2,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.01
3,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.01
4,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.01
...,...,...,...,...,...,...,...,...,...
3883,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.01
3884,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.01
3885,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.01
3886,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.01


In [ ]:
result_1.columns = ['0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9']

In [ ]:
def to_zero(x):
  if x < 0:
    return 0
  else:
    return x
    
result_1['0.1'] = result_1['0.1'].apply(to_zero)
result_1['0.2'] = result_1['0.2'].apply(to_zero)
result_1['0.3'] = result_1['0.3'].apply(to_zero)
result_1['0.4'] = result_1['0.4'].apply(to_zero)
result_1['0.5'] = result_1['0.5'].apply(to_zero)
result_1['0.6'] = result_1['0.6'].apply(to_zero)
result_1['0.7'] = result_1['0.7'].apply(to_zero)
result_1['0.8'] = result_1['0.8'].apply(to_zero)
result_1['0.9'] = result_1['0.9'].apply(to_zero)

In [ ]:
!pip install eli5
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(cb_model,n_iter=5, random_state=17).fit(X_valid_1, Y_valid_1)
eli5.show_weights(perm, top=100, feature_names = X_valid_1.columns.tolist())

Weight,Feature
0.0422 ± 0.0020,TARGET_day6
0.0255 ± 0.0007,Sun_DNI_hour_day6
0.0183 ± 0.0012,Sun_DHI_day1
0.0183 ± 0.0012,DHI_day6
0.0172 ± 0.0011,Sun_DHI_DNI_plus_day5
0.0166 ± 0.0012,Sun_TARGET_per_day5
0.0166 ± 0.0014,DHI_day1
0.0157 ± 0.0013,DHI_DNI_day6
0.0138 ± 0.0010,RH_day3
0.0133 ± 0.0005,DHI_day4


In [ ]:
1/3 + 1/2 + 1/1

1.8333333333333333

In [ ]:
1/3/(1/3 + 1/2 + 1/1) + 1/2/(1/3 + 1/2 + 1/1) + 1/1/(1/3 + 1/2 + 1/1)

1.0

In [ ]:
1/2/(1/3 + 1/2 + 1/1)

0.27272727272727276

In [ ]:
1/1/(1/3 + 1/2 + 1/1)

0.5454545454545455

In [ ]:
loss1 = 1.182626
loss2 = 1.12002
loss3 = 1.20042

In [ ]:
1/loss1/(1/loss1 + 1/loss2 + 1/loss3) + 1/loss2/(1/loss1 + 1/loss2 + 1/loss3) + 1/loss3/(1/loss1 + 1/loss2 + 1/loss3)

1.0

In [ ]:
weight1 = 1/loss1/(1/loss1 + 1/loss2 + 1/loss3)
weight2 = 1/loss2/(1/loss1 + 1/loss2 + 1/loss3)
weight3 = 1/loss3/(1/loss1 + 1/loss2 + 1/loss3)

print(weight1, weight2, weight3)
print(weight1+weight2+weight3)

0.32883118815576495 0.3472119361474792 0.32395687569675585
1.0
